# Imports

In [1]:
import gc
import glob
import os
import pathlib
from collections import defaultdict
from functools import partial
from itertools import product

import pandas as pd

# import dask.dataframe as pd
import plotly.express as px

# import ray
from IPython.display import Image
from tqdm.auto import tqdm, trange

# from tqdm import write
from tqdm.contrib.concurrent import process_map

# import modin.pandas as pd


# ray.init()

# Split the data

Put the data in folder `data`. Extract every file in the folder with the same name. If needed, adapt the path.

In [2]:
tsv_paths = sorted(
    glob.glob("data/NSDUH_Versions/*/*/*.tsv")
    + glob.glob("data/NSDUH_Versions/*/*/*/*.tsv")
)
tsv_paths

['data/NSDUH_Versions/NSDUH_Versions/NSDUH-2002-2014-DS0001-bndl-data-tsv/NSDUH-2002-2014-DS0001-data/NSDUH_2002_2014_Tab.tsv',
 'data/NSDUH_Versions/NSDUH_Versions/NSDUH-2002-2015-DS0001-bndl-data-tsv/NSDUH_2002_2015_Tab.tsv',
 'data/NSDUH_Versions/NSDUH_Versions/NSDUH-2002-2017-DS0001-bndl-data-tsv/NSDUH_2002_2017_Tab.tsv',
 'data/NSDUH_Versions/NSDUH_Versions/NSDUH-2002-2018-DS0001-bndl-data-tsv/NSDUH_2002_2018_Tab.tsv',
 'data/NSDUH_Versions/NSDUH_Versions/NSDUH-2002-2019-DS0001-bndl-data-tsv/NSDUH_2002_2019_Tab.tsv']

In [3]:
data_path = "data/NSDUN_versions"
pathlib.Path(data_path).mkdir(exist_ok=True)

# def dump(args):
#     v_path, year, data, header = args
#     with open(os.path.join(v_path, f"NSDUN_{year}.tsv"), "w") as f:
#         f.writelines([header] + data)


# for v, tsv_path in enumerate(tqdm(tsv_paths)):
#     v_path = os.path.join(data_path, f"V{v}")
#     pathlib.Path(v_path).mkdir(exist_ok=True)
#     with open(tsv_path, "r") as f:
#         tsv = f.readlines()

#     header = tsv[0]
#     tsv = tsv[1:]
#     tsvs = defaultdict(list)
#     for row in tsv:
#         tsvs[row[:4]].append(row)
#     process_map(dump, [(v_path, year, data, header) for year, data in tsvs.items()])

#     del tsv
#     del tsvs
#     del row
#     del header
#     gc.collect()

In [4]:
data_path = "data/NSDUN_versions2"
pathlib.Path(data_path).mkdir(exist_ok=True)


for v, tsv_path in enumerate(tqdm(tsv_paths)):
    v_path = os.path.join(data_path, f"V{v}")
    pathlib.Path(v_path).mkdir(exist_ok=True)

    tqdm.write("Loading data")
    df = pd.read_csv(tsv_path, sep="\t", skipinitialspace=True, col=['AGE', 'YEAR'])
    tqdm.write("Data loaded")

    for year, df_ in tqdm(df.groupby("YEAR")):
        path = os.path.join(v_path, f"NSDUN_{year}.pickle")
        df_.to_pickle(path, index=False)

    del df
    gc.collect()

  0%|          | 0/5 [00:00<?, ?it/s]

Loading data


KeyboardInterrupt: 

# Try all combinations

In [ ]:
tsv_paths = [
    [os.path.join(data_path, f"V{v}", f"NSDUN_{year}") + ".tsv" for v in range(0, 5)]
    for year in list(range(2002, 2015))
]

In [ ]:
tsv_paths

In [ ]:
with open(tsv_paths[0][2]) as f:
    bad_text = f.readlines()

In [ ]:
bad_text_csv = [line.split("\t") for line in tqdm(bad_text)]

In [ ]:
bad_columns = [row[2083:2086] for row in bad_text_csv]

In [ ]:
hm = [(int(a), b, int(c)) for a, b, c in bad_columns[1:]]

In [ ]:
ttwork_cols = ["WRKIDST2", "WRKIDSY2"]
filter_cols = ["YEAR", "AGE2", "JBSTATR2"]
survey_cols = ["ANALREC"]
cols = survey_cols + filter_cols

In [ ]:
good = pd.read_csv(tsv_paths[0][1], sep="\t", skipinitialspace=True, usecols=cols)
bad = pd.read_csv(tsv_paths[0][2], sep="\t", skipinitialspace=True, usecols=cols)

In [ ]:
bad_full.columns[2084]

In [ ]:
bad_full = pd.read_csv(tsv_paths[0][2], sep="\t", skipinitialspace=True)

In [ ]:
import numpy as np

In [ ]:
bad2 = pd.read_csv(tsv_paths[0][2], sep="\t", dtype={"GQTYPE": np.int64})

In [ ]:
bad2

In [ ]:
bad_full.iloc[:, 2084].dtype

In [ ]:
good

In [ ]:
dfs = [
    process_map(
        partial(pd.read_csv, sep="\t", skipinitialspace=True, usecols=cols), paths
    )
    for paths in tqdm(tsv_paths)
]

In [ ]:
dfs[0][3].iloc[:, 1864]

In [ ]:
for dfs_ in tqdm(dfs):
    for df in dfs_:
        df["AGE2"] = pd.to_numeric(df["AGE2"].str.split().str[0])

In [ ]:
from itertools import product

In [ ]:
rez = {}

N = sum(1 for _ in product(*tsv_paths))
for paths, dfs_ in tqdm(zip(product(*tsv_paths), product(*dfs)), total=N):
    n1, n2 = 0, 0
    for df in dfs_:
        n1 += len(df[df["AGE2"].str.split().str[0] <= 10])
        n2 += len(df[df["AGE2"].str.split().str[0] < 10])

    rez[paths] = (n1, n2)

In [ ]:
# dfs = process_map(
#     partial(pd.read_csv, sep="\t", skipinitialspace=True, usecols=cols), tsv_paths
# )
data_path = "data/NSDUN_versions"
if not os.path.exists(data_path):
    os.mkdir(data_path)

for v, tsv_path in enumerate(tqdm(tsv_paths)):
    v_path = os.path.join(data_path, f"V{v}")
    if not os.path.exists(v_path):
        os.mkdir(v_path)
    df = pd.read_csv(tsv_paths[0], sep="\t", skipinitialspace=True, engine="python")
    for year in trange(2002, 2015):
        df[df["YEAR"] == year].to_pickle(os.path.join(v_path, f"NSDUN_{year}.pkl"))

    del df

/home/astadnik/.virtualenvs/masters/lib/python3.10/site-packages/dask/dataframe/io/csv.py:182: DtypeWarning: Columns (1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769,1770,1771,1772,1773,1774,1775,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,18